### Tensorflow version 1.4

Note on tensorflow version:

> This notebook works with tensorflow 1.4. It uses the Dataset API of *tf.data*. In previous versions, the Dataset API was in *tf.contrib.data*. Although I did not test it, it may be possible to use this notebook with tensorflow 1.3 by replacing *tf.data* by *tf.contrib.data*.

In [11]:
import tensorflow as tf
import numpy as np

In [12]:
height = 5
width = 5

# Create random files containing raw matrix of shape 5x5
for i in range(2):
    for j in range(3):
        # Create matrix with fake data
        matrix = np.zeros((height,width)) + i + (j*10)
        # Save matrix as raw float32 file
        matrix.astype('float32').tofile('data/file_' + str(i) + '_' + str(j) + '.raw')

In [13]:
# Check one of the generated files
matrix = np.fromfile('data/file_1_1.raw', dtype=np.float32)
matrix = matrix.reshape((height,width))
matrix

array([[ 11.,  11.,  11.,  11.,  11.],
       [ 11.,  11.,  11.,  11.,  11.],
       [ 11.,  11.,  11.,  11.,  11.],
       [ 11.,  11.,  11.,  11.,  11.],
       [ 11.,  11.,  11.,  11.,  11.]], dtype=float32)

In [14]:
# Create parser
# Args: filenames
# Returns: tensor containing read and decoded element
def _parse_data(filename0, filename1, label):
    # Read and decode first file
    matrix0 = tf.read_file(filename0)
    matrix0 = tf.decode_raw(matrix0, out_type=tf.float32)
    matrix0 = tf.reshape(matrix0, [height,width])
    
    # Read and decode second file
    matrix1 = tf.read_file(filename1)
    matrix1 = tf.decode_raw(matrix1, out_type=tf.float32)
    matrix1 = tf.reshape(matrix1, [height,width])
    
    # Stack the two elements together
    X = tf.stack([matrix0, matrix1])
    
    # Get label (you could implement more complex logic here if needed)
    y = label
    
    return X, y

In [15]:
# Get the filenames lists
filenames0 = ['data/file_0_0.raw', 'data/file_0_1.raw', 'data/file_0_2.raw']
filenames1 = ['data/file_1_0.raw', 'data/file_1_1.raw', 'data/file_1_2.raw']

# Create tensorflow constant containing the filenames
tf_filenames0 = tf.constant(filenames0)
tf_filenames1 = tf.constant(filenames1)

# Create labels dataset
labels = tf.constant([15, 25, 10])

# Create datasets for each filenames list
dataset = tf.data.Dataset.from_tensor_slices((tf_filenames0, tf_filenames1, labels))

dataset = dataset.map(_parse_data)

# create TensorFlow Iterator object
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:

    # initialize the iterator on the training data
    sess.run(init_op)

    # get each element of the training dataset until the end is reached
    while True:
        try:
            elem = sess.run(next_element)
            print(elem)
        except tf.errors.OutOfRangeError:
            print("End of dataset.")
            break


AttributeError: module 'tensorflow' has no attribute 'data'